In [ ]:
import os
from crewai import Agent, Task, Crew
from langchain_anthropic import ChatAnthropic
from crewai_tools import SerperDevTool
from langchain_community.tools.tavily_search import TavilySearchResults
import pandas as pd
import matplotlib.pyplot as plt


# SETUP API KEYS

In [ ]:
os.environ['ANTHROPIC_API_KEY'] = 'your_anthropic_api_key_here'
os.environ['TAVILY_API_KEY'] = 'your_tavily_api_key_here'


# CONFIGURE CLAUDE 3 LLM

In [ ]:
llm = ChatAnthropic(
    model_name="claude-3-sonnet-20240229",
    temperature=0.3,
    max_tokens=2000,
)

# Web search tool

In [ ]:
search_tool = TavilySearchResults(k=5)

# Define agents with roles and backstories & tasks

In [ ]:
web_search_agent = Agent(
    role='Web Search & Aggregation Agent',
    goal='Query job platforms and gather raw AI/ML job posting data from MENA-focused job boards like LinkedIn, Bayt, Wuzzuf, and Glassdoor',
    backstory='Experienced web crawler and data miner specialized in extracting real-time job data across diverse platforms',
    tools=[SerperDevTool(api_key=api_key)],
    llm=llm,
    verbose=True
)

data_extraction_agent = Agent(
    role='Job Posting Parsing Agent',
    goal='Parse AI/ML job postings to extract job title, required skills, job location, company name, and any available salary information',
    backstory='Natural language processing expert skilled in converting unstructured job descriptions into structured records',
    tools=[SerperDevTool(api_key=api_key)],
    llm=llm
)

trend_analysis_agent = Agent(
    role='AI/ML Job Market Trend Analyst',
    goal='Identify high-demand job titles, frequently mentioned skills, location concentrations, and provide demand-based job ranking',
    backstory='Labor market analyst with specialization in emerging tech workforce patterns and predictive hiring trends',
    tools=[],
    llm=llm
)

report_writer_agent = Agent(
    role='AI/ML Labor Insights Report Writer',
    goal='Generate a clean and professional report (Markdown or PDF) summarizing AI/ML hiring trends in the MENA region',
    backstory='Former technical journalist with expertise in presenting labor market data with clarity and impact',
    tools=[],
    llm=llm
)

# Define the tasks

web_search_task = Task(
    description='Search LinkedIn, Bayt, Wuzzuf, and Glassdoor for AI/ML job postings in MENA countries and aggregate the raw job data',
    expected_output='Raw dataset containing job postings with basic metadata (e.g., post title, description, platform, URL)',
    agent=web_search_agent,
    output_file='raw_ai_ml_job_postings.json'
)

data_extraction_task = Task(
    description='Parse the raw job data to extract structured fields: job title, required skills, company, location, salary (if available)',
    expected_output='Structured JSON or CSV dataset with consistent fields across postings',
    agent=data_extraction_agent,
    output_file='structured_job_data.csv'
)

trend_analysis_task = Task(
    description='Analyze structured job data to find the top 10 most common job titles, most in-demand skills, geographic distribution, and rank jobs by demand',
    expected_output='Analytical summary including job title frequency, skill trends, country breakdowns, and a demand ranking table',
    agent=trend_analysis_agent
)

report_writing_task = Task(
    description='Generate a well-formatted report (Markdown or PDF) summarizing:\n1. Top 10 AI/ML roles\n2. Key required skills\n3. Country-wise job distribution\n4. Notable trends or shifts in hiring',
    expected_output='Final report titled "AI/ML Job Trends in MENA - Q2 2025" with charts and structured sections',
    agent=report_writer_agent
)


# Configure crew

In [ ]:
crew = Crew(
    agents=[web_search_agent, data_extraction_agent, trend_analysis_agent, report_writer_agent],
    tasks=[web_search_task, data_extraction_task, trend_analysis_task, report_writing_task],
    verbose=2,
    memory=True
)

results = crew.kickoff()

# Saving Report

In [ ]:
print("\n==== KEY FINDINGS ====")
print(results[:1500])  # Preview first 1500 chars

# Save full report
with open("MENA_AI_Talent_Report.md", "w") as f:
    f.write(results)

# visualization

In [ ]:
def create_visualizations():
    # Sample data (replace with actual analysis results)
    data = {
        'Country': ['UAE', 'Saudi Arabia', 'Egypt', 'Qatar'],
        'ML_Jobs': [120, 85, 65, 45],
        'Avg_Salary($)': [95000, 82000, 45000, 88000]
    }
    df = pd.DataFrame(data)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.bar(df['Country'], df['ML_Jobs'], color='royalblue')
    plt.title('AI/ML Job Openings by Country')
    
    plt.subplot(1, 2, 2)
    plt.plot(df['Country'], df['Avg_Salary($)'], marker='o', color='green', linewidth=3)
    plt.title('Average Salaries (USD)')
    plt.tight_layout()
    plt.savefig('mena_ai_jobs.png', dpi=300)

create_visualizations()

print("\nReport and visualizations generated successfully!")